# StatArb Exploration Notebook
Sélectionne des paires, trace spread & z-score.


In [ ]:
from pathlib import Path
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
from src.config import load_params
from src.universe import load_universe
from src.data import ensure_universe, get_price_series
from src.pairs import all_pairs_from_universe, score_pairs, select_top_pairs
from src.stats import spread_series, zscore, ols_hedge_ratio
params = load_params(); tickers = load_universe(); ensure_universe(params)
root_dir = Path(params['data']['root_dir'])
price_map = {}
for t in tickers:
    dfpl = get_price_series(root_dir, t).select(['date','close']).sort('date')
    price_map[t] = dfpl.to_pandas().set_index('date')
pairs = all_pairs_from_universe(tickers)
scored = score_pairs(price_map, pairs, params['lookbacks']['corr_days'], params['lookbacks']['coint_days'])
top = select_top_pairs(scored, params['selection']['min_corr'], params['selection']['max_half_life_days'], params['selection']['pval_coint'], 10)
top.head()


In [ ]:
a, b = top.loc[0, 'a'], top.loc[0, 'b']
dfa = price_map[a]['close']
dfb = price_map[b]['close']
alpha, beta = ols_hedge_ratio(dfa, dfb)
spr = spread_series(dfa, dfb, alpha, beta)
z = zscore(spr, params['lookbacks']['zscore_days'])
fig = plt.figure(); spr.plot(); plt.title(f'Spread {a}/{b}'); plt.show()
fig2 = plt.figure(); z.plot();
plt.axhline(params['thresholds']['entry_z']); plt.axhline(-params['thresholds']['entry_z']);
plt.axhline(params['thresholds']['exit_z']); plt.axhline(-params['thresholds']['exit_z']);
plt.title('Z-score et seuils'); plt.show()
